In [ ]:
import os

# Define paths
train_images_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/train/images"
train_labels_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/train/labels"
test_images_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/test/images"
test_labels_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/test/labels"
    "train": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/train/yolo_aa_labels",
    "val":   "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/val/yolo_aa_labels",
    "test":  "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/test/yolo_aa_labels"
def collect_image_label_pairs(images_dir, labels_dir):
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.png') or f.endswith('.jpg')]
    pairs = []
    for img in image_files:
        label = os.path.splitext(img)[0] + '.txt'
        label_path = os.path.join(labels_dir, label)
        img_path = os.path.join(images_dir, img)
        if os.path.exists(label_path):
            pairs.append((img_path, label_path))
        else:
            print(f"Label missing for image: {img}")
    return pairs

train_pairs = collect_image_label_pairs(train_images_dir, train_labels_dir)
test_pairs = collect_image_label_pairs(test_images_dir, test_labels_dir)

print(f"Train image-label pairs: {len(train_pairs)}")
print(f"Test image-label pairs: {len(test_pairs)}")

In [ ]:
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd
def get_latlon_corners(center_lat, center_lon, points, img_size=128, gsd=10):
    latlons = []
    for x, y in points:
        dx = x - img_size / 2
        dy = y - img_size / 2
        dx_m = dx * gsd
        dy_m = dy * gsd
        dlat = dy_m / 111320
        dlon = dx_m / (111320 * np.cos(np.deg2rad(center_lat)))
        lat = center_lat + dlat
        lon = center_lon + dlon
        latlons.append((lon, lat))
    return latlons

def folder_to_gdf(folder, split):
    data = []
    for fname in os.listdir(folder):
        if fname.endswith('.txt'):
            image_name = fname.replace('.txt', '.png')
            lat_str, lon_str = image_name.replace('.png', '').split('_')
            center_lat = float(lat_str)
            center_lon = float(lon_str)
            label_path = os.path.join(folder, fname)
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 9:
                        continue
                    class_id = int(parts[0])
                    coords = list(map(float, parts[1:]))
                    points = np.array(coords).reshape(4,2) * 128
                    latlons = get_latlon_corners(center_lat, center_lon, points)
                    poly = Polygon(latlons)
                    data.append({
                        'image_name': image_name,
                        'class': class_id,
                        'geometry': poly,
                        'split': split
                    })
    return gpd.GeoDataFrame(data, crs="EPSG:4326")

folders = {
    "train": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/train/yolo_aa_labels",
    "val":   "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/val/yolo_aa_labels",
    "test":  "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/test/yolo_aa_labels"
}

gdfs = [folder_to_gdf(folders[split], split) for split in ["train", "val", "test"]]
gdf_all = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="EPSG:4326")




In [1]:
import os

def get_image_label_lists(images_dir, labels_dir):
    image_files = sorted([os.path.join(images_dir, f) for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg'))])
    label_files = sorted([os.path.join(labels_dir, f) for f in os.listdir(labels_dir) if f.endswith('.txt')])
    return image_files, label_files

train_images_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/train/images"
train_labels_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/train/labels"
test_images_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/test/images"
test_labels_dir = "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/test/labels"

train_image_list, train_label_list = get_image_label_lists(train_images_dir, train_labels_dir)
test_image_list, test_label_list = get_image_label_lists(test_images_dir, test_labels_dir)

print("Train images:", len(train_image_list))
print("Train labels:", len(train_label_list))
print("Test images:", len(test_image_list))
print("Test labels:", len(test_label_list))

# Optionally, print a sample
#print(train_image_list[:3])
#print(train_label_list[:3])


Train images: 63810
Train labels: 63810
Test images: 14882
Test labels: 14882


In [2]:
import os

aa_label_dirs = {
    "train": "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/train/yolo_aa_labels",
    "val":   "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/val/yolo_aa_labels",
    "test":  "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/sentinel/final_data_neurips_2025/test/yolo_aa_labels"
}

label_lists = {}

for split, dir_path in aa_label_dirs.items():
    label_lists[split] = sorted([
        os.path.join(dir_path, f)
        for f in os.listdir(dir_path)
        if f.endswith('.txt')
    ])
    print(f"{split.capitalize()} AA label files: {len(label_lists[split])}")

# Example access:
# print(label_lists['train'][:3])


Train AA label files: 47214
Val AA label files: 15738
Test AA label files: 15738


In [3]:
import os

def get_base_names(file_list, ext='.txt'):
    return set(os.path.splitext(os.path.basename(f))[0] for f in file_list if f.endswith(ext))

# Read base names from your AA label dirs
aa_base_names = {}
for split, dir_path in aa_label_dirs.items():
    files = [f for f in os.listdir(dir_path) if f.endswith('.txt')]
    aa_base_names[split] = set(os.path.splitext(f)[0] for f in files)

# Read base names from your original label dirs
orig_label_dirs = {
    "train": "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/train/labels",
    "test": "/home/suruchi.hardaha/cosmos/ijcai_2025_data/symlinked_AA_DATA/train_afghanistan_bangladesh_india_test_pakistan/test/labels"
}
orig_base_names = {}
for split, dir_path in orig_label_dirs.items():
    files = [f for f in os.listdir(dir_path) if f.endswith('.txt')]
    orig_base_names[split] = set(os.path.splitext(f)[0] for f in files)

# Compare base names for each split
for split in ["train", "test"]:
    aa_names = aa_base_names.get(split, set())
    orig_names = orig_base_names.get(split, set())
    print(f"{split.capitalize()} - AA labels: {len(aa_names)}, Orig labels: {len(orig_names)}")
    common = aa_names & orig_names
    only_in_aa = aa_names - orig_names
    only_in_orig = orig_names - aa_names
    print(f"  Common base names: {len(common)}")
    print(f"  Only in AA labels: {len(only_in_aa)}")
    print(f"  Only in Orig labels: {len(only_in_orig)}")
    # Optionally print some differences
    # print(f"  Only in AA: {list(only_in_aa)[:5]}")
    # print(f"  Only in Orig: {list(only_in_orig)[:5]}")


Train - AA labels: 47214, Orig labels: 63810
  Common base names: 5143
  Only in AA labels: 42071
  Only in Orig labels: 58667
Test - AA labels: 15738, Orig labels: 14882
  Common base names: 0
  Only in AA labels: 15738
  Only in Orig labels: 14882
